In [1]:
import pandas as pd
import folium

In [2]:
df = pd.read_csv("datasets V2AIX/df_v4.csv")

In [3]:
df_trajet = pd.read_excel("datasets V2AIX/df_trips.xlsx")

In [4]:
# Supprimer la limite d'affichage des colonnes
pd.set_option('display.max_columns', None)

df.head()

,Unnamed: 0,recording_timestamp_nsec,message_header_protocol_version,message_header_message_id,message_header_station_id_value,message_cam_generation_delta_time_value,message_cam_cam_parameters_basic_container_station_type_value,message_cam_cam_parameters_basic_container_reference_position_latitude_value,message_cam_cam_parameters_basic_container_reference_position_longitude_value,message_cam_cam_parameters_basic_container_reference_position_position_confidence_ellipse_semi_major_confidence_value,message_cam_cam_parameters_basic_container_reference_position_position_confidence_ellipse_semi_minor_confidence_value,message_cam_cam_parameters_basic_container_reference_position_position_confidence_ellipse_semi_major_orientation_value,message_cam_cam_parameters_basic_container_reference_position_altitude_altitude_value_value,message_cam_cam_parameters_basic_container_reference_position_altitude_altitude_confidence_value,message_cam_cam_parameters_high_frequency_container_choice,message_cam_cam_parameters_high_frequency_container_basic_vehicle_container_high_frequency_heading_heading_value_value,message_cam_cam_parameters_high_frequency_container_basic_vehicle_container_high_frequency_heading_heading_confidence_value,message_cam_cam_parameters_high_frequency_container_basic_vehicle_container_high_frequency_speed_speed_value_value,message_cam_cam_parameters_high_frequency_container_basic_vehicle_container_high_frequency_speed_speed_confidence_value,message_cam_cam_parameters_high_frequency_container_basic_vehicle_container_high_frequency_drive_direction_value,message_cam_cam_parameters_high_frequency_container_basic_vehicle_container_high_frequency_vehicle_length_vehicle_length_value_value,message_cam_cam_parameters_high_frequency_container_basic_vehicle_container_high_frequency_vehicle_length_vehicle_length_confidence_indication_value,message_cam_cam_parameters_high_frequency_container_basic_vehicle_container_high_frequency_vehicle_width_value,message_cam_cam_parameters_high_frequency_container_basic_vehicle_container_high_frequency_longitudinal_acceleration_longitudinal_acceleration_value_value,message_cam_cam_parameters_high_frequency_container_basic_vehicle_container_high_frequency_longitudinal_acceleration_longitudinal_acceleration_confidence_value,message_cam_cam_parameters_high_frequency_container_basic_vehicle_container_high_frequency_curvature_curvature_value_value,message_cam_cam_parameters_high_frequency_container_basic_vehicle_container_high_frequency_curvature_curvature_confidence_value,message_cam_cam_parameters_high_frequency_container_basic_vehicle_container_high_frequency_curvature_calculation_mode_value,message_cam_cam_parameters_high_frequency_container_basic_vehicle_container_high_frequency_yaw_rate_yaw_rate_value_value,message_cam_cam_parameters_high_frequency_container_basic_vehicle_container_high_frequency_yaw_rate_yaw_rate_confidence_value,message_cam_cam_parameters_high_frequency_container_basic_vehicle_container_high_frequency_acceleration_control_value,message_cam_cam_parameters_high_frequency_container_basic_vehicle_container_high_frequency_acceleration_control_bits_unused,message_cam_cam_parameters_high_frequency_container_basic_vehicle_container_high_frequency_acceleration_control_is_present,message_cam_cam_parameters_high_frequency_container_basic_vehicle_container_high_frequency_lane_position_value,message_cam_cam_parameters_high_frequency_container_basic_vehicle_container_high_frequency_lane_position_is_present,message_cam_cam_parameters_high_frequency_container_basic_vehicle_container_high_frequency_steering_wheel_angle_steering_wheel_angle_value_value,message_cam_cam_parameters_high_frequency_container_basic_vehicle_container_high_frequency_steering_wheel_angle_steering_wheel_angle_confidence_value,message_cam_cam_parameters_high_frequency_container_basic_vehicle_container_high_frequency_steering_wheel_angle_is_present,message_cam_cam_parameters_high_frequency_container_basic_vehicle_container_high_f

In [5]:
# df_sample = df.sample(100)
# df_sample.to_excel("df_sample.xlsx")

In [6]:
df = df.rename(columns={"message_header_station_id_value": "station_id"})

In [7]:
df["timestamp"] = pd.to_datetime(df["timestamp"], errors="coerce")

In [8]:
df["pseudonym_id_fcmz"] = df["station_id"].astype(str) + "_0"
df["ht_flag"] = False  # sert à marquer les véhicules intégrés en fin de cycle

In [9]:
import numpy as np

# Coordonnées de Aachen
aachen_lat = 50.7753
aachen_lon = 6.0839
radius_km = 5

# Conversion du rayon en degrés
lat_range = radius_km / 111
lon_range = radius_km / (111 * np.cos(np.radians(aachen_lat)))

# Filtrage des messages CAM autour d’Aachen
df = df[
    (df["latitude"].between(aachen_lat - lat_range, aachen_lat + lat_range)) &
    (df["longitude"].between(aachen_lon - lon_range, aachen_lon + lon_range))
].copy()

# Calcul surface et densité locale (si needed)
area_aachen_km2 = np.pi * radius_km**2
n_points = len(df)
print(f"Surface couverte : {area_aachen_km2:.2f} km²")
print(f"{n_points} CAMs dans un rayon de {radius_km} km autour d’Aachen.")

Surface couverte : 78.54 km²
90951 CAMs dans un rayon de 5 km autour d’Aachen.


In [10]:
from geopy.distance import geodesic
from collections import defaultdict
import pandas as pd
from datetime import timedelta
from tqdm import tqdm

def detect_fcmz_cohorts(df, T=15, radius_m=200, k=4, cooldown_s=30):
    """
    Détecte des cohortes FCMZ en filtrant selon un temps minimal entre deux participations par station_id.

    Args:
        df (pd.DataFrame): base contenant au moins 'timestamp', 'station_id', 'latitude', 'longitude'
        T (int): durée d’un slot temporel (en secondes)
        radius_m (float): rayon spatial en mètres
        k (int): nombre minimal de véhicules pour former une cohorte
        cooldown_s (int): temps minimal entre deux participations (en secondes)

    Returns:
        List[dict]: Liste des cohortes détectées
    """
    df = df.copy()
    df["timestamp"] = pd.to_datetime(df["timestamp"])
    df = df.sort_values("timestamp").reset_index(drop=True)

    t_min = df["timestamp"].min()
    t_max = df["timestamp"].max()
    time_slots = pd.date_range(start=t_min, end=t_max, freq=f"{T}s")

    # Initialisation du dictionnaire last_seen à une valeur réaliste
    default_time = t_min - timedelta(seconds=cooldown_s)
    last_seen = defaultdict(lambda: default_time)

    cohorts = []

    for t_start in tqdm(time_slots):
        t_end = t_start + pd.Timedelta(seconds=T)
        slot_df = df[(df["timestamp"] >= t_start) & (df["timestamp"] < t_end)]

        if len(slot_df) < k:
            continue

        coords = slot_df[["station_id", "latitude", "longitude"]].values

        for i in range(len(coords)):
            sid_i, lat_i, lon_i = coords[i]
            if (t_start - last_seen[sid_i]).total_seconds() < cooldown_s:
                continue

            nearby = [sid_i]

            for j in range(len(coords)):
                if i == j:
                    continue
                sid_j, lat_j, lon_j = coords[j]
                if (t_start - last_seen[sid_j]).total_seconds() < cooldown_s:
                    continue
                dist = geodesic((lat_i, lon_i), (lat_j, lon_j)).meters
                if dist <= radius_m:
                    nearby.append(sid_j)

            if len(set(nearby)) >= k:
                cohorts.append({
                    "start_time": t_start,
                    "station_ids": list(set(nearby))
                })
                for sid in nearby:
                    last_seen[sid] = t_start

    return cohorts

In [11]:
# cohorts_paper = detect_fcmz_cohorts(df, T=5, radius_m=250, k=4, cooldown_s=30)
# print(f"Nombre de cohortes FCMZ détectées (avec filtre) : {len(cohorts_paper)}")

In [12]:
cohorts = detect_fcmz_cohorts(df, T=15, radius_m=250, k=4, cooldown_s=30)
print(f"Nombre de cohortes FCMZ détectées (avec filtre) : {len(cohorts)}")

100%|██████████| 5622/5622 [08:53<00:00, 10.54it/s]  

Nombre de cohortes FCMZ détectées (avec filtre) : 40


In [13]:
# cohorts_3 = detect_fcmz_cohorts(df, T=15, radius_m=250, k=3, cooldown_s=30)
# print(f"Nombre de cohortes FCMZ détectées (avec filtre) : {len(cohorts_3)}")

In [14]:
import numpy as np

def apply_flickering(df, cohorts, T=5, n=3, p=0.5, pseudonym_col="pseudonym_id_fcmz"):
    """
    Applique le schéma de flickering FCMZ :
    - Pour chaque cohorte, pendant n slots de durée T, chaque véhicule diffuse avec probabilité p.
    - À la fin, tous changent de pseudonyme.

    Args:
        df (pd.DataFrame): Base avec les messages CAM.
        cohorts (List[dict]): Liste de cohortes détectées avec 'start_time' et 'station_ids'.
        T (int): Durée de chaque slot (en secondes).
        n (int): Nombre de slots intermédiaires (sans le dernier de changement).
        p (float): Probabilité de diffuser pendant le flickering.
        pseudonym_col (str): Nom de la colonne de pseudonymes simulés.

    Returns:
        pd.DataFrame: DataFrame modifié avec les colonnes :
            - pseudonym_id_fcmz
            - silent_period_fcmz (True si message non diffusé dans slot)
    """
    df = df.copy()
    df["timestamp"] = pd.to_datetime(df["timestamp"])
    df[pseudonym_col] = df["station_id"].astype(str)
    df["silent_period_fcmz"] = False

    for idx, cohort in enumerate(cohorts):
        t_start = cohort["start_time"]
        station_ids = set(cohort["station_ids"])

        # Générer un pseudonyme unique par station pour cette cohorte
        new_pids = {sid: f"fcmz_{idx}_{sid}" for sid in station_ids}

        for slot in range(1, n+1):
            t_slot_start = t_start + pd.Timedelta(seconds=slot * T)
            t_slot_end = t_slot_start + pd.Timedelta(seconds=T)

            for sid in station_ids:
                mask = (
                    (df["station_id"] == sid) &
                    (df["timestamp"] >= t_slot_start) &
                    (df["timestamp"] < t_slot_end)
                )
                # Diffusion probabiliste
                broadcast = np.random.rand() < p
                if not broadcast:
                    df.loc[mask, "silent_period_fcmz"] = True
                else:
                    df.loc[mask, pseudonym_col] = new_pids[sid]

        # Slot final : changement effectif de pseudonyme
        final_start = t_start + pd.Timedelta(seconds=(n+1)*T)
        mask_final = (
            (df["station_id"].isin(station_ids)) &
            (df["timestamp"] >= final_start)
        )
        for sid in station_ids:
            df.loc[mask_final & (df["station_id"] == sid), pseudonym_col] = new_pids[sid]

    return df

In [15]:
df = apply_flickering(df, cohorts=cohorts, T=15, n=3, p=0.5)

In [16]:
from geopy.distance import geodesic

def mark_hiding_tail(df, cohorts, T=5, n=3, radius_m=250):
    """
    Marque les véhicules voisins de la cohorte à t+nT comme faisant partie de la hiding tail.

    Args:
        df (pd.DataFrame): Données contenant station_id, timestamp, latitude, longitude
        cohorts (List[dict]): Liste de cohortes avec start_time et station_ids
        T (int): Durée d’un slot (en secondes)
        n (int): Nombre de slots avant le changement final
        radius_m (float): Rayon de proximité pour inclusion dans hiding tail

    Returns:
        pd.DataFrame: Base annotée avec une colonne 'hiding_tail' (bool)
    """
    df = df.copy()
    df["timestamp"] = pd.to_datetime(df["timestamp"])
    df["hiding_tail"] = False

    for cohort in cohorts:
        t_ht = cohort["start_time"] + pd.Timedelta(seconds=n * T)
        window_df = df[(df["timestamp"] >= t_ht) & (df["timestamp"] < t_ht + pd.Timedelta(seconds=T))]

        # Récupère la position des membres de la cohorte à cet instant
        cohort_df = window_df[window_df["station_id"].isin(cohort["station_ids"])]
        cohort_pos = cohort_df.groupby("station_id")[["latitude", "longitude"]].mean().values

        # Marque les voisins proches dans la même fenêtre
        for _, row in window_df.iterrows():
            sid, lat, lon = row["station_id"], row["latitude"], row["longitude"]
            for c_lat, c_lon in cohort_pos:
                dist = geodesic((lat, lon), (c_lat, c_lon)).meters
                if dist <= radius_m:
                    df.loc[row.name, "hiding_tail"] = True
                    break

    return df


In [17]:
df = mark_hiding_tail(df, cohorts=cohorts, T=15, n=3, radius_m=250)

In [18]:
def finalize_fcmz_pseudonyms(df, cohorts, T=5, n=3, pseudonym_col="pseudonym_id_fcmz"):
    """
    Applique un changement de pseudonyme final à tous les participants FCMZ (cohorte + hiding tail)
    au moment t + (n+1)T.

    Args:
        df (pd.DataFrame): Base CAM contenant les colonnes station_id, timestamp, hiding_tail.
        cohorts (List[dict]): Liste de cohortes FCMZ (avec start_time et station_ids).
        T (int): Durée d’un slot.
        n (int): Nombre de slots intermédiaires (hors changement).
        pseudonym_col (str): Nom de la colonne de pseudonymes simulés.

    Returns:
        pd.DataFrame: df mis à jour avec des pseudonymes modifiés.
    """
    df = df.copy()
    df["timestamp"] = pd.to_datetime(df["timestamp"])
    
    for idx, cohort in enumerate(cohorts):
        t_change = cohort["start_time"] + pd.Timedelta(seconds=(n + 1) * T)
        all_participants = set(cohort["station_ids"])

        # Ajouter les hiding tail détectés dans la fenêtre cible
        tail_window = df[
            (df["timestamp"] >= t_change) &
            (df["timestamp"] < t_change + pd.Timedelta(seconds=T)) &
            (df["hiding_tail"])
        ]["station_id"].unique()

        all_participants.update(tail_window)

        for sid in all_participants:
            mask = (df["station_id"] == sid) & (df["timestamp"] >= t_change)
            df.loc[mask, pseudonym_col] = f"fcmz_final_{idx}_{sid}"

    return df


In [19]:
df = finalize_fcmz_pseudonyms(df, cohorts=cohorts, T=15, n=3)


In [20]:
from geopy.distance import geodesic

# Assurer le tri par trajet et timestamp
df = df.sort_values(by=['trip_id', 'timestamp']).copy()

# Initialiser la colonne 'distance'
df['distance'] = 0.0

# Calculer la distance cumulée par trajet
for trip_id, group in df.groupby('trip_id'):
    distances = [0.0]
    coords = list(zip(group['latitude'], group['longitude']))
    for i in range(1, len(coords)):
        dist = geodesic(coords[i - 1], coords[i]).meters
        distances.append(distances[-1] + dist)
    df.loc[group.index, 'distance'] = distances

In [21]:
print(df_trajet['trip_id'].dtype)

object


In [22]:
print(df['trip_id'].dtype)

object


In [23]:
df['date'] = df['timestamp'].dt.date

# Agrégation par pseudonyme simulé
df_trajet_simule = df.groupby('pseudonym_id_fcmz').agg({
    'timestamp': ['min', 'max'],
    'latitude': ['first', 'last'],
    'longitude': ['first', 'last'],
    'distance': 'max',
    'speed': 'mean',  # proxy si pas d'autre info de type véhicule
}).reset_index()

# Aplatir les colonnes multi-index
df_trajet_simule.columns = [
    'pseudonym_id', 'start_time', 'end_time',
    'start_latitude', 'end_latitude',
    'start_longitude', 'end_longitude',
    'distance_m', 'mean_speed'
]

# Ajout de variables dérivées
df_trajet_simule["trip_duration_seconds"] = (df_trajet_simule["end_time"] - df_trajet_simule["start_time"]).dt.total_seconds()
df_trajet_simule["trip_duration_minutes"] = df_trajet_simule["trip_duration_seconds"] / 60
df_trajet_simule["distance_km"] = df_trajet_simule["distance_m"] / 1000

# Extraire le trip_id depuis pseudonym_id (format "123456_0")
# df_trajet_simule['trip_id'] = df_trajet_simule['pseudonym_id'].str.extract(r'^(\d+)_')[0].astype(int)
# df_trajet_simule['trip_id'] = df_trajet_simule['trip_id'].astype(int)
# df_trajet['trip_id'] = df_trajet['trip_id'].astype(int)

# Extraire station_id associé à chaque pseudonym_id_simulated
station_map = df.groupby("pseudonym_id_fcmz")["station_id"].agg(lambda x: x.mode().iloc[0]).reset_index()

# Renommer pour correspondre
station_map.columns = ["pseudonym_id", "station_id"]

# Fusionner avec df_trajet_simule
df_trajet_simule = df_trajet_simule.merge(station_map, on="pseudonym_id", how="left")

# Extraire la correspondance station_id → model_id depuis df_trajet
station_to_model = df_trajet[['station_id', 'model_id']].drop_duplicates()

# Fusionner avec df_trajet_simule sur station_id
df_trajet_simule = df_trajet_simule.merge(station_to_model, on='station_id', how='left')



In [24]:
df_trajet_simule = df_trajet_simule.merge(df[["pseudonym_id_fcmz", "trip_id"]].drop_duplicates(), left_on="pseudonym_id", right_on="pseudonym_id_fcmz", how="left")

In [25]:
df_trajet_simule["model_id"].value_counts(dropna=False)

model_id
NaN    1048
1.0     411
3.0      82
2.0      76
4.0       9
6.0       9
7.0       5
5.0       1
Name: count, dtype: int64

In [26]:
station_simule = set(df_trajet_simule["station_id"].unique())
station_ref = set(df_trajet["station_id"].unique())

missing_stations = station_simule - station_ref
print(f"{len(missing_stations)} station_id dans df_trajet_simule ne sont pas dans df_trajet")

974 station_id dans df_trajet_simule ne sont pas dans df_trajet


In [27]:
# Nombre de station_id distincts par pseudonym_id
station_per_pseudonym = df_trajet_simule.groupby("station_id")["pseudonym_id"].nunique()

# Compter ceux qui sont associés à un seul véhicule
unique_station_count = (station_per_pseudonym == 1).sum()

# Proportion de pseudonymes associés à un seul véhicule
pourcentage_identifiants_vehicule = 100 * unique_station_count / len(station_per_pseudonym)
print("Proportion de pseudonym_id associés à un seul véhicule :", np.round(pourcentage_identifiants_vehicule, 2), "%")

Proportion de pseudonym_id associés à un seul véhicule : 94.37 %


In [28]:
# Agréger par station_id
agg_by_station = df_trajet_simule.groupby("station_id").agg({
    "distance_m": "sum",
    "pseudonym_id": "nunique"
})

# Garder seulement les véhicules ayant parcouru au total ≥ 800m
agg_by_station = agg_by_station[agg_by_station["distance_m"] >= 800]

# Calculer proportion de ceux qui ont eu un seul pseudonyme
proportion = 100 * (agg_by_station["pseudonym_id"] == 1).sum() / len(agg_by_station)
print("Proportion de véhicules avec un seul pseudonym_id (≥800m cumulés) :", np.round(proportion, 2), "%")

Proportion de véhicules avec un seul pseudonym_id (≥800m cumulés) : 62.96 %


In [29]:
# Garder seulement les véhicules ayant parcouru au total ≥ 1500m
agg_by_station = agg_by_station[agg_by_station["distance_m"] >= 1500]

# Calculer proportion de ceux qui ont eu un seul pseudonyme
proportion = 100 * (agg_by_station["pseudonym_id"] == 1).sum() / len(agg_by_station)
print("Proportion de véhicules avec un seul pseudonym_id (≥1500m cumulés) :", np.round(proportion, 2), "%")

Proportion de véhicules avec un seul pseudonym_id (≥1500m cumulés) : 66.67 %


In [43]:
df_trajet_simule = df_trajet_simule.sort_values("start_time").reset_index(drop=True)
df_trajet_simule["segment_index"] = df_trajet_simule.index

In [44]:
import numpy as np

def compute_bearing(lat1, lon1, lat2, lon2):
    # Calcul de la direction entre deux points (en degrés)
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlon = lon2 - lon1

    x = np.sin(dlon) * np.cos(lat2)
    y = np.cos(lat1) * np.sin(lat2) - (np.sin(lat1) * np.cos(lat2) * np.cos(dlon))
    bearing = np.degrees(np.arctan2(x, y))
    return (bearing + 360) % 360

# Appliquer sur le DataFrame
df_trajet_simule["direction"] = compute_bearing(
    df_trajet_simule["start_latitude"],
    df_trajet_simule["start_longitude"],
    df_trajet_simule["end_latitude"],
    df_trajet_simule["end_longitude"]
)

# Pour compatibilité avec l'algo
df_trajet_simule["start_direction"] = df_trajet_simule["direction"]
df_trajet_simule["end_direction"] = df_trajet_simule["direction"]


Relinking Naïf

In [50]:
from geopy.distance import geodesic

def relinking_score(row_A, row_B):
    # Distance en mètres entre la fin de A et le début de B
    spatial_dist = geodesic(
        (row_A["end_latitude"], row_A["end_longitude"]),
        (row_B["start_latitude"], row_B["start_longitude"])
    ).meters

    # Temps écoulé en secondes entre A et B
    temporal_gap = (row_B["start_time"] - row_A["end_time"]).total_seconds()

    # Score combiné (plus bas = plus probable)
    # pondération arbitraire : 1 mètre ≈ 1 seconde ici
    if temporal_gap < 0 or temporal_gap > 600:  # exclusion >10 min ou B commence avant A se termine
        return np.inf

    return spatial_dist + temporal_gap  # ou une pondération personnalisée


In [51]:
best_links = []

for i, row_A in df_trajet_simule.iterrows():
    candidates = df_trajet_simule[df_trajet_simule["start_time"] > row_A["end_time"]]
    
    best_score = np.inf
    best_match_id = None
    
    for _, row_B in candidates.iterrows():
        score = relinking_score(row_A, row_B)
        if score < best_score:
            best_score = score
            best_match_id = row_B["pseudonym_id"]
    
    if best_match_id is not None:
        best_links.append({
            "source": row_A["pseudonym_id"],
            "target": best_match_id,
            "score": best_score
        })

In [52]:
df_links = pd.DataFrame(best_links)

In [53]:
from collections import defaultdict

# Créer un graphe orienté : source → target
link_dict = dict(zip(df_links["source"], df_links["target"]))

# Stocker les chaînes reconstruites
chains = []
visited = set()

for source in link_dict:
    if source in visited:
        continue

    chain = [source]
    visited.add(source)
    current = source

    while current in link_dict:
        next_pseudo = link_dict[current]
        if next_pseudo in visited or next_pseudo == current:
            break
        chain.append(next_pseudo)
        visited.add(next_pseudo)
        current = next_pseudo

    chains.append(chain)


In [54]:
reconstructed_trip_map = {}
for i, chain in enumerate(chains):
    for pseudo_id in chain:
        reconstructed_trip_map[pseudo_id] = f"retrip_{i}"


In [55]:
df_eval = df_trajet_simule[["pseudonym_id", "trip_id"]].copy()
df_eval["retrip_id"] = df_eval["pseudonym_id"].map(reconstructed_trip_map)

In [56]:
# Pour chaque trip_id, combien de retrip_id différents ?
trip_fragmentation = df_eval.groupby("trip_id")["retrip_id"].nunique()

# Pourcentage de trip_id qui ont été entièrement reconstitués
nb_reconstitués = (trip_fragmentation == 1).sum()
pourcentage = 100 * nb_reconstitués / len(trip_fragmentation)

print(f"Réidentification complète de trajets : {pourcentage:.2f}%")

Réidentification complète de trajets : 96.47%


In [57]:
# Vraies paires successives dans le même trip_id
true_links = []

df_sorted = df_trajet_simule.sort_values(["trip_id", "start_time"])

for _, group in df_sorted.groupby("trip_id"):
    pseudos = group["pseudonym_id"].tolist()
    true_links.extend([(pseudos[i], pseudos[i+1]) for i in range(len(pseudos)-1)])


In [58]:
pred_links = list(zip(df_links["source"], df_links["target"]))

In [59]:
true_set = set(true_links)
pred_set = set(pred_links)

tp = len(true_set & pred_set)  # vrais positifs : GPA a bien relié deux segments du même trip_id
fp = len(pred_set - true_set)  # faux positifs : GPA a relié deux pseudonymes de trip_ids différents
fn = len(true_set - pred_set)  # faux négatifs : GPA a raté un lien réel

precision = tp / (tp + fp) if (tp + fp) > 0 else 0
recall = tp / (tp + fn) if (tp + fn) > 0 else 0
idf1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

print(f"Relinking (pairwise):")
print(f"Precision : {precision:.2%}")
print(f"Recall    : {recall:.2%}")
print(f"IDF1      : {idf1:.2%}")

Relinking (pairwise):
Precision : 6.94%
Recall    : 78.42%
IDF1      : 12.76%


In [63]:
def evaluate_linking_attack_pairwise(df_trajet_simule, df_links):
    from sklearn.metrics import adjusted_rand_score

    # Étape 1 : reconstruction des chaînes
    link_dict = dict(zip(df_links["source"], df_links["target"]))
    chains = []
    visited = set()

    for source in link_dict:
        if source in visited:
            continue
        chain = [source]
        visited.add(source)
        current = source
        while current in link_dict:
            next_pseudo = link_dict[current]
            if next_pseudo in visited or next_pseudo == current:
                break
            chain.append(next_pseudo)
            visited.add(next_pseudo)
            current = next_pseudo
        chains.append(chain)

    reconstructed_trip_map = {}
    for i, chain in enumerate(chains):
        for pseudo_id in chain:
            reconstructed_trip_map[pseudo_id] = f"retrip_{i}"

    df_eval = df_trajet_simule[["pseudonym_id", "trip_id", "start_time"]].copy()
    df_eval["retrip_id"] = df_eval["pseudonym_id"].map(reconstructed_trip_map)

    # Métrique 1 : % de trajets entièrement réidentifiés
    fragmentation = df_eval.groupby("trip_id")["retrip_id"].nunique()
    fully_reidentified = (fragmentation == 1).sum()
    percent_fully = 100 * fully_reidentified / len(fragmentation)

    # Métrique 2 : IDF1 pairwise
    df_sorted = df_eval.sort_values(["trip_id", "start_time"])
    true_links = []
    for _, group in df_sorted.groupby("trip_id"):
        pseudos = group["pseudonym_id"].tolist()
        true_links.extend([(pseudos[i], pseudos[i+1]) for i in range(len(pseudos)-1)])

    pred_links = list(zip(df_links["source"], df_links["target"]))
    true_set = set(true_links)
    pred_set = set(pred_links)

    tp = len(true_set & pred_set)
    fp = len(pred_set - true_set)
    fn = len(true_set - pred_set)

    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    idf1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

    # Métrique 3 : ID switches
    id_switches = 0
    for _, group in df_eval.sort_values(["trip_id", "start_time"]).groupby("trip_id"):
        rids = group["retrip_id"].tolist()
        switches = sum([rids[i] != rids[i-1] for i in range(1, len(rids))])
        id_switches += switches

    # Métrique 4 : faux liens
    false_links = fp

    # Métrique 5 : ARI
    ari = adjusted_rand_score(df_eval["trip_id"], df_eval["retrip_id"].fillna('unlinked'))

    return pd.DataFrame([{
        "%_trips_fully_reidentified": percent_fully,
        "false_links": false_links,
        "id_switches": id_switches,
        "ARI": ari,
        "IDF1": 100 * idf1
    }])


In [66]:
df_eval = df_trajet_simule[["pseudonym_id", "trip_id"]].copy()
df_eval["retrip_id"] = df_eval["pseudonym_id"].map(reconstructed_trip_map)
df_eval["start_time"] = df_trajet_simule["start_time"]  # ajouter le temps pour les ordres

In [67]:
results = evaluate_linking_attack_pairwise(df_trajet_simule, df_links)
print(results.T)

                                      0
%_trips_fully_reidentified    96.471372
false_links                 1461.000000
id_switches                   65.000000
ARI                            0.008532
IDF1                          12.755998


Relinking Seuil

In [68]:
def relinking_score(row_A, row_B, time_limit=30, distance_limit=1500):
    # Distance en mètres
    spatial_dist = geodesic(
        (row_A["end_latitude"], row_A["end_longitude"]),
        (row_B["start_latitude"], row_B["start_longitude"])
    ).meters

    # Temps écoulé en secondes
    temporal_gap = (row_B["start_time"] - row_A["end_time"]).total_seconds()

    # Rejeter si contraintes non respectées
    if temporal_gap < 0 or temporal_gap > time_limit or spatial_dist > distance_limit:
        return np.inf

    # Score combiné (on peut ajuster la pondération)
    return spatial_dist + temporal_gap


In [69]:
best_links = []

for i, row_A in df_trajet_simule.iterrows():
    candidates = df_trajet_simule[df_trajet_simule["start_time"] > row_A["end_time"]]

    best_score = np.inf
    best_match_id = None

    for _, row_B in candidates.iterrows():
        score = relinking_score(row_A, row_B)
        if score < best_score:
            best_score = score
            best_match_id = row_B["pseudonym_id"]

    if best_score < np.inf:
        best_links.append({
            "source": row_A["pseudonym_id"],
            "target": best_match_id,
            "score": best_score
        })


In [70]:
df_links = pd.DataFrame(best_links)

In [71]:
from collections import defaultdict

# Créer un graphe orienté : source → target
link_dict = dict(zip(df_links["source"], df_links["target"]))

# Stocker les chaînes reconstruites
chains = []
visited = set()

for source in link_dict:
    if source in visited:
        continue

    chain = [source]
    visited.add(source)
    current = source

    while current in link_dict:
        next_pseudo = link_dict[current]
        if next_pseudo in visited or next_pseudo == current:
            break
        chain.append(next_pseudo)
        visited.add(next_pseudo)
        current = next_pseudo

    chains.append(chain)


In [72]:
reconstructed_trip_map = {}
for i, chain in enumerate(chains):
    for pseudo_id in chain:
        reconstructed_trip_map[pseudo_id] = f"retrip_{i}"


In [73]:
df_eval = df_trajet_simule[["pseudonym_id", "trip_id"]].copy()
df_eval["retrip_id"] = df_eval["pseudonym_id"].map(reconstructed_trip_map)

In [74]:
# Pour chaque trip_id, combien de retrip_id différents ?
trip_fragmentation = df_eval.groupby("trip_id")["retrip_id"].nunique()

# Pourcentage de trip_id qui ont été entièrement reconstitués
nb_reconstitués = (trip_fragmentation == 1).sum()
pourcentage = 100 * nb_reconstitués / len(trip_fragmentation)

print(f"Réidentification complète de trajets : {pourcentage:.2f}%")

Réidentification complète de trajets : 62.38%


In [75]:
# Vraies paires successives dans le même trip_id
true_links = []

df_sorted = df_trajet_simule.sort_values(["trip_id", "start_time"])

for _, group in df_sorted.groupby("trip_id"):
    pseudos = group["pseudonym_id"].tolist()
    true_links.extend([(pseudos[i], pseudos[i+1]) for i in range(len(pseudos)-1)])


In [76]:
pred_links = list(zip(df_links["source"], df_links["target"]))

In [77]:
true_set = set(true_links)
pred_set = set(pred_links)

tp = len(true_set & pred_set)  # vrais positifs : GPA a bien relié deux segments du même trip_id
fp = len(pred_set - true_set)  # faux positifs : GPA a relié deux pseudonymes de trip_ids différents
fn = len(true_set - pred_set)  # faux négatifs : GPA a raté un lien réel

precision = tp / (tp + fp) if (tp + fp) > 0 else 0
recall = tp / (tp + fn) if (tp + fn) > 0 else 0
idf1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

print(f"Relinking (pairwise):")
print(f"Precision : {precision:.2%}")
print(f"Recall    : {recall:.2%}")
print(f"IDF1      : {idf1:.2%}")

Relinking (pairwise):
Precision : 13.19%
Recall    : 76.98%
IDF1      : 22.53%


In [78]:
def evaluate_linking_attack_pairwise(df_trajet_simule, df_links):
    from sklearn.metrics import adjusted_rand_score

    # Étape 1 : reconstruction des chaînes
    link_dict = dict(zip(df_links["source"], df_links["target"]))
    chains = []
    visited = set()

    for source in link_dict:
        if source in visited:
            continue
        chain = [source]
        visited.add(source)
        current = source
        while current in link_dict:
            next_pseudo = link_dict[current]
            if next_pseudo in visited or next_pseudo == current:
                break
            chain.append(next_pseudo)
            visited.add(next_pseudo)
            current = next_pseudo
        chains.append(chain)

    reconstructed_trip_map = {}
    for i, chain in enumerate(chains):
        for pseudo_id in chain:
            reconstructed_trip_map[pseudo_id] = f"retrip_{i}"

    df_eval = df_trajet_simule[["pseudonym_id", "trip_id", "start_time"]].copy()
    df_eval["retrip_id"] = df_eval["pseudonym_id"].map(reconstructed_trip_map)

    # Métrique 1 : % de trajets entièrement réidentifiés
    fragmentation = df_eval.groupby("trip_id")["retrip_id"].nunique()
    fully_reidentified = (fragmentation == 1).sum()
    percent_fully = 100 * fully_reidentified / len(fragmentation)

    # Métrique 2 : IDF1 pairwise
    df_sorted = df_eval.sort_values(["trip_id", "start_time"])
    true_links = []
    for _, group in df_sorted.groupby("trip_id"):
        pseudos = group["pseudonym_id"].tolist()
        true_links.extend([(pseudos[i], pseudos[i+1]) for i in range(len(pseudos)-1)])

    pred_links = list(zip(df_links["source"], df_links["target"]))
    true_set = set(true_links)
    pred_set = set(pred_links)

    tp = len(true_set & pred_set)
    fp = len(pred_set - true_set)
    fn = len(true_set - pred_set)

    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    idf1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

    # Métrique 3 : ID switches
    id_switches = 0
    for _, group in df_eval.sort_values(["trip_id", "start_time"]).groupby("trip_id"):
        rids = group["retrip_id"].tolist()
        switches = sum([rids[i] != rids[i-1] for i in range(1, len(rids))])
        id_switches += switches

    # Métrique 4 : faux liens
    false_links = fp

    # Métrique 5 : ARI
    ari = adjusted_rand_score(df_eval["trip_id"], df_eval["retrip_id"].fillna('unlinked'))

    return pd.DataFrame([{
        "%_trips_fully_reidentified": percent_fully,
        "false_links": false_links,
        "id_switches": id_switches,
        "ARI": ari,
        "IDF1": 100 * idf1
    }])


In [79]:
df_eval = df_trajet_simule[["pseudonym_id", "trip_id"]].copy()
df_eval["retrip_id"] = df_eval["pseudonym_id"].map(reconstructed_trip_map)
df_eval["start_time"] = df_trajet_simule["start_time"]  # ajouter le temps pour les ordres

In [80]:
results = evaluate_linking_attack_pairwise(df_trajet_simule, df_links)
print(results.T)

                                     0
%_trips_fully_reidentified   62.383489
false_links                 704.000000
id_switches                  62.000000
ARI                           0.001473
IDF1                         22.526316


Relinking prudent

In [81]:
from geopy.distance import geodesic

def relinking_score(row_A, row_B, time_limit=30, distance_limit=1500):
    # Spatio-temporel
    spatial_dist = geodesic(
        (row_A["end_latitude"], row_A["end_longitude"]),
        (row_B["start_latitude"], row_B["start_longitude"])
    ).meters
    temporal_gap = (row_B["start_time"] - row_A["end_time"]).total_seconds()

    if temporal_gap < 0 or temporal_gap > time_limit or spatial_dist > distance_limit:
        return np.inf  # Hors limite

    # Optionnels mais utiles : vitesse et direction
    speed_diff = abs(row_A["mean_speed"] - row_B["mean_speed"])
    direction_diff = abs((row_A["end_direction"] - row_B["start_direction"] + 180) % 360 - 180)

    # Bonus si modèle identique
    model_bonus = 0 if row_A["model_id"] == row_B["model_id"] else 200  # pénalité légère

    # Score pondéré (ajustable)
    score = spatial_dist + 2 * temporal_gap + 5 * speed_diff + 3 * direction_diff + model_bonus
    return score



In [82]:
best_links = []

for i, row_A in df_trajet_simule.iterrows():
    # Chercher les 5 suivants dans le temps
    candidates = df_trajet_simule[df_trajet_simule["start_time"] > row_A["end_time"]]
    candidates = candidates.nsmallest(5, "start_time")  # seulement 5 plus proches

    best_score = np.inf
    best_match_id = None

    for _, row_B in candidates.iterrows():
        score = relinking_score(row_A, row_B)
        if score < best_score:
            best_score = score
            best_match_id = row_B["pseudonym_id"]

    # On accepte le lien seulement s’il a un score réaliste (ex: < 1000)
    if best_score < 100:
        best_links.append({
            "source": row_A["pseudonym_id"],
            "target": best_match_id,
            "score": best_score
        })


In [83]:
df_links = pd.DataFrame(best_links)

In [84]:
from collections import defaultdict

# Créer un graphe orienté : source → target
link_dict = dict(zip(df_links["source"], df_links["target"]))

# Stocker les chaînes reconstruites
chains = []
visited = set()

for source in link_dict:
    if source in visited:
        continue

    chain = [source]
    visited.add(source)
    current = source

    while current in link_dict:
        next_pseudo = link_dict[current]
        if next_pseudo in visited or next_pseudo == current:
            break
        chain.append(next_pseudo)
        visited.add(next_pseudo)
        current = next_pseudo

    chains.append(chain)


In [85]:
reconstructed_trip_map = {}
for i, chain in enumerate(chains):
    for pseudo_id in chain:
        reconstructed_trip_map[pseudo_id] = f"retrip_{i}"


In [86]:
df_eval = df_trajet_simule[["pseudonym_id", "trip_id"]].copy()
df_eval["retrip_id"] = df_eval["pseudonym_id"].map(reconstructed_trip_map)

In [87]:
# Pour chaque trip_id, combien de retrip_id différents ?
trip_fragmentation = df_eval.groupby("trip_id")["retrip_id"].nunique()

# Pourcentage de trip_id qui ont été entièrement reconstitués
nb_reconstitués = (trip_fragmentation == 1).sum()
pourcentage = 100 * nb_reconstitués / len(trip_fragmentation)

print(f"Réidentification complète de trajets : {pourcentage:.2f}%")

Réidentification complète de trajets : 1.66%


In [88]:
# Vraies paires successives dans le même trip_id
true_links = []

df_sorted = df_trajet_simule.sort_values(["trip_id", "start_time"])

for _, group in df_sorted.groupby("trip_id"):
    pseudos = group["pseudonym_id"].tolist()
    true_links.extend([(pseudos[i], pseudos[i+1]) for i in range(len(pseudos)-1)])


In [89]:
pred_links = list(zip(df_links["source"], df_links["target"]))

In [90]:
true_set = set(true_links)
pred_set = set(pred_links)

tp = len(true_set & pred_set)  # vrais positifs : GPA a bien relié deux segments du même trip_id
fp = len(pred_set - true_set)  # faux positifs : GPA a relié deux pseudonymes de trip_ids différents
fn = len(true_set - pred_set)  # faux négatifs : GPA a raté un lien réel

precision = tp / (tp + fp) if (tp + fp) > 0 else 0
recall = tp / (tp + fn) if (tp + fn) > 0 else 0
idf1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

print(f"Relinking (pairwise):")
print(f"Precision : {precision:.2%}")
print(f"Recall    : {recall:.2%}")
print(f"IDF1      : {idf1:.2%}")

Relinking (pairwise):
Precision : 69.05%
Recall    : 20.86%
IDF1      : 32.04%


In [91]:
def evaluate_linking_attack_pairwise(df_trajet_simule, df_links):
    from sklearn.metrics import adjusted_rand_score

    # Étape 1 : reconstruction des chaînes
    link_dict = dict(zip(df_links["source"], df_links["target"]))
    chains = []
    visited = set()

    for source in link_dict:
        if source in visited:
            continue
        chain = [source]
        visited.add(source)
        current = source
        while current in link_dict:
            next_pseudo = link_dict[current]
            if next_pseudo in visited or next_pseudo == current:
                break
            chain.append(next_pseudo)
            visited.add(next_pseudo)
            current = next_pseudo
        chains.append(chain)

    reconstructed_trip_map = {}
    for i, chain in enumerate(chains):
        for pseudo_id in chain:
            reconstructed_trip_map[pseudo_id] = f"retrip_{i}"

    df_eval = df_trajet_simule[["pseudonym_id", "trip_id", "start_time"]].copy()
    df_eval["retrip_id"] = df_eval["pseudonym_id"].map(reconstructed_trip_map)

    # Métrique 1 : % de trajets entièrement réidentifiés
    fragmentation = df_eval.groupby("trip_id")["retrip_id"].nunique()
    fully_reidentified = (fragmentation == 1).sum()
    percent_fully = 100 * fully_reidentified / len(fragmentation)

    # Métrique 2 : IDF1 pairwise
    df_sorted = df_eval.sort_values(["trip_id", "start_time"])
    true_links = []
    for _, group in df_sorted.groupby("trip_id"):
        pseudos = group["pseudonym_id"].tolist()
        true_links.extend([(pseudos[i], pseudos[i+1]) for i in range(len(pseudos)-1)])

    pred_links = list(zip(df_links["source"], df_links["target"]))
    true_set = set(true_links)
    pred_set = set(pred_links)

    tp = len(true_set & pred_set)
    fp = len(pred_set - true_set)
    fn = len(true_set - pred_set)

    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    idf1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

    # Métrique 3 : ID switches
    id_switches = 0
    for _, group in df_eval.sort_values(["trip_id", "start_time"]).groupby("trip_id"):
        rids = group["retrip_id"].tolist()
        switches = sum([rids[i] != rids[i-1] for i in range(1, len(rids))])
        id_switches += switches

    # Métrique 4 : faux liens
    false_links = fp

    # Métrique 5 : ARI
    ari = adjusted_rand_score(df_eval["trip_id"], df_eval["retrip_id"].fillna('unlinked'))

    return pd.DataFrame([{
        "%_trips_fully_reidentified": percent_fully,
        "false_links": false_links,
        "id_switches": id_switches,
        "ARI": ari,
        "IDF1": 100 * idf1
    }])

df_eval = df_trajet_simule[["pseudonym_id", "trip_id"]].copy()
df_eval["retrip_id"] = df_eval["pseudonym_id"].map(reconstructed_trip_map)
df_eval["start_time"] = df_trajet_simule["start_time"]  # ajouter le temps pour les ordres
results = evaluate_linking_attack_pairwise(df_trajet_simule, df_links)
print(results.T)

                                     0
%_trips_fully_reidentified    1.664447
false_links                  13.000000
id_switches                 113.000000
ARI                          -0.000079
IDF1                         32.044199


In [92]:
from geopy.distance import geodesic
from scipy.stats import norm
import numpy as np
import pandas as pd

# Fonction de probabilité de liaison
def nnpda_score(row_A, row_B, sigma_d=200, sigma_t=10):
    # Distance et temps
    spatial_dist = geodesic(
        (row_A["end_latitude"], row_A["end_longitude"]),
        (row_B["start_latitude"], row_B["start_longitude"])
    ).meters
    temporal_gap = (row_B["start_time"] - row_A["end_time"]).total_seconds()

    if temporal_gap < 0:
        return 0  # pas possible

    # Probabilités gaussiennes (PDF)
    p_d = norm.pdf(spatial_dist, loc=0, scale=sigma_d)
    p_t = norm.pdf(temporal_gap, loc=0, scale=sigma_t)

    return p_d * p_t

# Relinking NNPDA
best_links_nnpda = []

for i, row_A in df_trajet_simule.iterrows():
    candidates = df_trajet_simule[df_trajet_simule["start_time"] > row_A["end_time"]]
    candidates = candidates.nsmallest(10, "start_time")

    best_score = 0
    best_match_id = None

    for _, row_B in candidates.iterrows():
        score = nnpda_score(row_A, row_B)
        if score > best_score:
            best_score = score
            best_match_id = row_B["pseudonym_id"]

    # Seuil de probabilité minimum pour considérer le lien
    if best_score > 1e-5:
        best_links_nnpda.append({
            "source": row_A["pseudonym_id"],
            "target": best_match_id,
            "score": best_score
        })

df_links_nnpda = pd.DataFrame(best_links_nnpda)

# Chainer les liens comme avant
link_dict = dict(zip(df_links_nnpda["source"], df_links_nnpda["target"]))
chains = []
visited = set()

for source in link_dict:
    if source in visited:
        continue
    chain = [source]
    visited.add(source)
    current = source
    while current in link_dict:
        next_id = link_dict[current]
        if next_id in visited or next_id == current:
            break
        chain.append(next_id)
        visited.add(next_id)
        current = next_id
    chains.append(chain)

reconstructed_map = {pid: f"retrip_nnpda_{i}" for i, chain in enumerate(chains) for pid in chain}
df_eval = df_trajet_simule[["pseudonym_id", "trip_id"]].copy()
df_eval["retrip_id"] = df_eval["pseudonym_id"].map(reconstructed_map)

# Évaluation IDF1
true_links = []
df_sorted = df_trajet_simule.sort_values(["trip_id", "start_time"])
for _, group in df_sorted.groupby("trip_id"):
    pseudos = group["pseudonym_id"].tolist()
    true_links.extend([(pseudos[i], pseudos[i+1]) for i in range(len(pseudos)-1)])

pred_links = list(zip(df_links_nnpda["source"], df_links_nnpda["target"]))
true_set = set(true_links)
pred_set = set(pred_links)

tp = len(true_set & pred_set)
fp = len(pred_set - true_set)
fn = len(true_set - pred_set)

precision = tp / (tp + fp) if (tp + fp) else 0
recall = tp / (tp + fn) if (tp + fn) else 0
idf1 = 2 * precision * recall / (precision + recall) if (precision + recall) else 0

print(f"NNPDA Relinking:")
print(f"Precision : {precision:.2%}")
print(f"Recall    : {recall:.2%}")
print(f"IDF1      : {idf1:.2%}")


NNPDA Relinking:
Precision : 22.44%
Recall    : 66.19%
IDF1      : 33.52%


In [93]:
# Pour chaque trip_id, combien de retrip_id différents ?
trip_fragmentation = df_eval.groupby("trip_id")["retrip_id"].nunique()

# Pourcentage de trip_id qui ont été entièrement reconstitués
nb_reconstitués = (trip_fragmentation == 1).sum()
pourcentage = 100 * nb_reconstitués / len(trip_fragmentation)

print(f"Réidentification complète de trajets : {pourcentage:.2f}%")

Réidentification complète de trajets : 28.83%


In [94]:
def evaluate_linking_attack_pairwise(df_trajet_simule, df_links):
    from sklearn.metrics import adjusted_rand_score

    # Étape 1 : reconstruction des chaînes
    link_dict = dict(zip(df_links["source"], df_links["target"]))
    chains = []
    visited = set()

    for source in link_dict:
        if source in visited:
            continue
        chain = [source]
        visited.add(source)
        current = source
        while current in link_dict:
            next_pseudo = link_dict[current]
            if next_pseudo in visited or next_pseudo == current:
                break
            chain.append(next_pseudo)
            visited.add(next_pseudo)
            current = next_pseudo
        chains.append(chain)

    reconstructed_trip_map = {}
    for i, chain in enumerate(chains):
        for pseudo_id in chain:
            reconstructed_trip_map[pseudo_id] = f"retrip_{i}"

    df_eval = df_trajet_simule[["pseudonym_id", "trip_id", "start_time"]].copy()
    df_eval["retrip_id"] = df_eval["pseudonym_id"].map(reconstructed_trip_map)

    # Métrique 1 : % de trajets entièrement réidentifiés
    fragmentation = df_eval.groupby("trip_id")["retrip_id"].nunique()
    fully_reidentified = (fragmentation == 1).sum()
    percent_fully = 100 * fully_reidentified / len(fragmentation)

    # Métrique 2 : IDF1 pairwise
    df_sorted = df_eval.sort_values(["trip_id", "start_time"])
    true_links = []
    for _, group in df_sorted.groupby("trip_id"):
        pseudos = group["pseudonym_id"].tolist()
        true_links.extend([(pseudos[i], pseudos[i+1]) for i in range(len(pseudos)-1)])

    pred_links = list(zip(df_links["source"], df_links["target"]))
    true_set = set(true_links)
    pred_set = set(pred_links)

    tp = len(true_set & pred_set)
    fp = len(pred_set - true_set)
    fn = len(true_set - pred_set)

    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    idf1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

    # Métrique 3 : ID switches
    id_switches = 0
    for _, group in df_eval.sort_values(["trip_id", "start_time"]).groupby("trip_id"):
        rids = group["retrip_id"].tolist()
        switches = sum([rids[i] != rids[i-1] for i in range(1, len(rids))])
        id_switches += switches

    # Métrique 4 : faux liens
    false_links = fp

    # Métrique 5 : ARI
    ari = adjusted_rand_score(df_eval["trip_id"], df_eval["retrip_id"].fillna('unlinked'))

    return pd.DataFrame([{
        "%_trips_fully_reidentified": percent_fully,
        "false_links": false_links,
        "id_switches": id_switches,
        "ARI": ari,
        "IDF1": 100 * idf1
    }])

df_eval = df_trajet_simule[["pseudonym_id", "trip_id"]].copy()
df_eval["retrip_id"] = df_eval["pseudonym_id"].map(reconstructed_trip_map)
df_eval["start_time"] = df_trajet_simule["start_time"]  # ajouter le temps pour les ordres
results = evaluate_linking_attack_pairwise(df_trajet_simule, df_links)
print(results.T)

                                     0
%_trips_fully_reidentified    1.664447
false_links                  13.000000
id_switches                 113.000000
ARI                          -0.000079
IDF1                         32.044199


In [95]:
from pykalman import KalmanFilter
from geopy.distance import geodesic
import numpy as np

def kalman_predict_end_position(row):
    """
    Applique un filtre de Kalman simple (1D par axe) pour estimer la position future.
    Utilise la vitesse moyenne et direction pour projeter la position.
    """
    # Simplification : on projette linéairement en utilisant la moyenne de vitesse
    lat, lon = row["end_latitude"], row["end_longitude"]
    speed = row["mean_speed"] / 3.6  # km/h → m/s
    duration = 5  # secondes dans le futur

    # Projection très simple : distance = v * t, déplacé en ligne droite vers l'avant
    # Ce n'est pas un vrai Kalman, mais un proxy si pas de mesures intermédiaires
    # (remplacé par un vrai Kalman plus bas si tu veux l'étendre avec observation)

    return lat, lon  # ici, juste la position réelle (pas de prédiction)

def relinking_score_kalman(row_A, row_B):
    pred_lat, pred_lon = kalman_predict_end_position(row_A)
    dist = geodesic((pred_lat, pred_lon), (row_B["start_latitude"], row_B["start_longitude"])).meters

    time_gap = (row_B["start_time"] - row_A["end_time"]).total_seconds()
    if time_gap < 0 or time_gap > 60:
        return np.inf

    speed_diff = abs(row_A["mean_speed"] - row_B["mean_speed"])
    return dist + 5 * speed_diff + 2 * time_gap

# Relinking
best_links = []

for i, row_A in df_trajet_simule.iterrows():
    candidates = df_trajet_simule[df_trajet_simule["start_time"] > row_A["end_time"]]
    candidates = candidates[candidates["start_time"] <= row_A["end_time"] + pd.Timedelta(seconds=60)]

    best_score = np.inf
    best_match_id = None

    for _, row_B in candidates.iterrows():
        score = relinking_score_kalman(row_A, row_B)
        if score < best_score:
            best_score = score
            best_match_id = row_B["pseudonym_id"]

    if best_score < 100:  # seuil réaliste
        best_links.append({
            "source": row_A["pseudonym_id"],
            "target": best_match_id,
            "score": best_score
        })

df_links = pd.DataFrame(best_links)

In [96]:
from collections import defaultdict

# Créer un graphe orienté : source → target
link_dict = dict(zip(df_links["source"], df_links["target"]))

# Stocker les chaînes reconstruites
chains = []
visited = set()

for source in link_dict:
    if source in visited:
        continue

    chain = [source]
    visited.add(source)
    current = source

    while current in link_dict:
        next_pseudo = link_dict[current]
        if next_pseudo in visited or next_pseudo == current:
            break
        chain.append(next_pseudo)
        visited.add(next_pseudo)
        current = next_pseudo

    chains.append(chain)


In [97]:
reconstructed_trip_map = {}
for i, chain in enumerate(chains):
    for pseudo_id in chain:
        reconstructed_trip_map[pseudo_id] = f"retrip_{i}"


In [98]:
df_eval = df_trajet_simule[["pseudonym_id", "trip_id"]].copy()
df_eval["retrip_id"] = df_eval["pseudonym_id"].map(reconstructed_trip_map)

In [99]:
# Pour chaque trip_id, combien de retrip_id différents ?
trip_fragmentation = df_eval.groupby("trip_id")["retrip_id"].nunique()

# Pourcentage de trip_id qui ont été entièrement reconstitués
nb_reconstitués = (trip_fragmentation == 1).sum()
pourcentage = 100 * nb_reconstitués / len(trip_fragmentation)

print(f"Réidentification complète de trajets : {pourcentage:.2f}%")

Réidentification complète de trajets : 21.50%


In [100]:
# Vraies paires successives dans le même trip_id
true_links = []

df_sorted = df_trajet_simule.sort_values(["trip_id", "start_time"])

for _, group in df_sorted.groupby("trip_id"):
    pseudos = group["pseudonym_id"].tolist()
    true_links.extend([(pseudos[i], pseudos[i+1]) for i in range(len(pseudos)-1)])


In [101]:
pred_links = list(zip(df_links["source"], df_links["target"]))

In [102]:
true_set = set(true_links)
pred_set = set(pred_links)

tp = len(true_set & pred_set)  # vrais positifs : GPA a bien relié deux segments du même trip_id
fp = len(pred_set - true_set)  # faux positifs : GPA a relié deux pseudonymes de trip_ids différents
fn = len(true_set - pred_set)  # faux négatifs : GPA a raté un lien réel

precision = tp / (tp + fp) if (tp + fp) > 0 else 0
recall = tp / (tp + fn) if (tp + fn) > 0 else 0
idf1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

print(f"Relinking (pairwise):")
print(f"Precision : {precision:.2%}")
print(f"Recall    : {recall:.2%}")
print(f"IDF1      : {idf1:.2%}")

Relinking (pairwise):
Precision : 33.66%
Recall    : 73.38%
IDF1      : 46.15%


In [103]:
def evaluate_linking_attack_pairwise(df_trajet_simule, df_links):
    from sklearn.metrics import adjusted_rand_score

    # Étape 1 : reconstruction des chaînes
    link_dict = dict(zip(df_links["source"], df_links["target"]))
    chains = []
    visited = set()

    for source in link_dict:
        if source in visited:
            continue
        chain = [source]
        visited.add(source)
        current = source
        while current in link_dict:
            next_pseudo = link_dict[current]
            if next_pseudo in visited or next_pseudo == current:
                break
            chain.append(next_pseudo)
            visited.add(next_pseudo)
            current = next_pseudo
        chains.append(chain)

    reconstructed_trip_map = {}
    for i, chain in enumerate(chains):
        for pseudo_id in chain:
            reconstructed_trip_map[pseudo_id] = f"retrip_{i}"

    df_eval = df_trajet_simule[["pseudonym_id", "trip_id", "start_time"]].copy()
    df_eval["retrip_id"] = df_eval["pseudonym_id"].map(reconstructed_trip_map)

    # Métrique 1 : % de trajets entièrement réidentifiés
    fragmentation = df_eval.groupby("trip_id")["retrip_id"].nunique()
    fully_reidentified = (fragmentation == 1).sum()
    percent_fully = 100 * fully_reidentified / len(fragmentation)

    # Métrique 2 : IDF1 pairwise
    df_sorted = df_eval.sort_values(["trip_id", "start_time"])
    true_links = []
    for _, group in df_sorted.groupby("trip_id"):
        pseudos = group["pseudonym_id"].tolist()
        true_links.extend([(pseudos[i], pseudos[i+1]) for i in range(len(pseudos)-1)])

    pred_links = list(zip(df_links["source"], df_links["target"]))
    true_set = set(true_links)
    pred_set = set(pred_links)

    tp = len(true_set & pred_set)
    fp = len(pred_set - true_set)
    fn = len(true_set - pred_set)

    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    idf1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

    # Métrique 3 : ID switches
    id_switches = 0
    for _, group in df_eval.sort_values(["trip_id", "start_time"]).groupby("trip_id"):
        rids = group["retrip_id"].tolist()
        switches = sum([rids[i] != rids[i-1] for i in range(1, len(rids))])
        id_switches += switches

    # Métrique 4 : faux liens
    false_links = fp

    # Métrique 5 : ARI
    ari = adjusted_rand_score(df_eval["trip_id"], df_eval["retrip_id"].fillna('unlinked'))

    return pd.DataFrame([{
        "%_trips_fully_reidentified": percent_fully,
        "false_links": false_links,
        "id_switches": id_switches,
        "ARI": ari,
        "IDF1": 100 * idf1
    }])

df_eval = df_trajet_simule[["pseudonym_id", "trip_id"]].copy()
df_eval["retrip_id"] = df_eval["pseudonym_id"].map(reconstructed_trip_map)
df_eval["start_time"] = df_trajet_simule["start_time"]  # ajouter le temps pour les ordres
results = evaluate_linking_attack_pairwise(df_trajet_simule, df_links)
print(results.T)

                                     0
%_trips_fully_reidentified   21.504660
false_links                 201.000000
id_switches                  62.000000
ARI                           0.000052
IDF1                         46.153846


In [104]:
import pandas as pd
import networkx as nx
from geopy.distance import geodesic

# Paramètres ajustables
max_time_gap = 30       # secondes
max_dist_km = 1.5       # kilomètres

# Créer un graphe dirigé
G = nx.DiGraph()

# Ajout des nœuds
for pseudo_id in df_trajet_simule["pseudonym_id"]:
    G.add_node(pseudo_id)

# Construction des arêtes entre segments compatibles
for i, row_A in df_trajet_simule.iterrows():
    for j, row_B in df_trajet_simule.iterrows():
        if row_A["pseudonym_id"] == row_B["pseudonym_id"]:
            continue

        # Condition temporelle
        delta_t = (row_B["start_time"] - row_A["end_time"]).total_seconds()
        if delta_t <= 0 or delta_t > max_time_gap:
            continue

        # Condition spatiale
        dist = geodesic(
            (row_A["end_latitude"], row_A["end_longitude"]),
            (row_B["start_latitude"], row_B["start_longitude"])
        ).km
        if dist > max_dist_km:
            continue

        # Ajout de l'arête
        G.add_edge(row_A["pseudonym_id"], row_B["pseudonym_id"])

# Extraire les composantes fortement connexes
components = list(nx.weakly_connected_components(G))

# Assigner un retrip_id à chaque composante
pseudo_to_retrip = {}
for i, comp in enumerate(components):
    for pseudo_id in comp:
        pseudo_to_retrip[pseudo_id] = f"retrip_{i}"

# Appliquer au DataFrame
df_trajet_simule["retrip_id"] = df_trajet_simule["pseudonym_id"].map(pseudo_to_retrip)


In [105]:
import pandas as pd
import numpy as np
import networkx as nx
from geopy.distance import geodesic
from tqdm import tqdm

# Fonction de score de lien (plus le score est bas, plus les points sont proches)
def linking_score(row_A, row_B, time_limit=60, distance_limit=1500):
    spatial_dist = geodesic(
        (row_A["end_latitude"], row_A["end_longitude"]),
        (row_B["start_latitude"], row_B["start_longitude"])
    ).meters
    temporal_gap = (row_B["start_time"] - row_A["end_time"]).total_seconds()

    if temporal_gap < 0 or temporal_gap > time_limit or spatial_dist > distance_limit:
        return np.inf

    # Score = distance + pondération temporelle
    return spatial_dist + 2 * temporal_gap

# Construction du graphe
G = nx.DiGraph()

for i, row_A in tqdm(df_trajet_simule.iterrows(), total=len(df_trajet_simule)):
    G.add_node(row_A["pseudonym_id"])
    candidates = df_trajet_simule[df_trajet_simule["start_time"] > row_A["end_time"]]
    candidates = candidates[candidates["start_time"] <= row_A["end_time"] + pd.Timedelta(seconds=60)]

    for _, row_B in candidates.iterrows():
        score = linking_score(row_A, row_B)
        if score < 100:  # seuil ajustable
            G.add_edge(row_A["pseudonym_id"], row_B["pseudonym_id"], weight=score)

# Détection des composantes fortement connexes (retraçage de trajets)
components = list(nx.weakly_connected_components(G))
reconstructed_trip_map = {}
for i, comp in enumerate(components):
    for pseudo in comp:
        reconstructed_trip_map[pseudo] = f"retrip_graph_{i}"

df_eval = df_trajet_simule[["pseudonym_id", "trip_id"]].copy()
df_eval["retrip_id"] = df_eval["pseudonym_id"].map(reconstructed_trip_map)

# Évaluation
trip_fragmentation = df_eval.groupby("trip_id")["retrip_id"].nunique()
nb_reconstitués = (trip_fragmentation == 1).sum()
pourcentage = 100 * nb_reconstitués / len(trip_fragmentation)

print(f"Réidentification complète de trajets (graph-based) : {pourcentage:.2f}%")

# Liens prédits
pred_links = set(G.edges())
true_links = []

df_sorted = df_trajet_simule.sort_values(["trip_id", "start_time"])
for _, group in df_sorted.groupby("trip_id"):
    pseudos = group["pseudonym_id"].tolist()
    true_links.extend([(pseudos[i], pseudos[i+1]) for i in range(len(pseudos)-1)])

true_set = set(true_links)
tp = len(true_set & pred_links)
fp = len(pred_links - true_set)
fn = len(true_set - pred_links)

precision = tp / (tp + fp) if (tp + fp) > 0 else 0
recall = tp / (tp + fn) if (tp + fn) > 0 else 0
idf1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

print(f"Relinking (graph-based):")
print(f"Precision : {precision:.2%}")
print(f"Recall    : {recall:.2%}")
print(f"IDF1      : {idf1:.2%}")


100%|██████████| 1641/1641 [00:02<00:00, 612.08it/s]


Réidentification complète de trajets (graph-based) : 99.53%
Relinking (graph-based):
Precision : 17.54%
Recall    : 79.14%
IDF1      : 28.72%


In [106]:
def evaluate_linking_attack_pairwise(df_trajet_simule, df_links):
    from sklearn.metrics import adjusted_rand_score

    # Étape 1 : reconstruction des chaînes
    link_dict = dict(zip(df_links["source"], df_links["target"]))
    chains = []
    visited = set()

    for source in link_dict:
        if source in visited:
            continue
        chain = [source]
        visited.add(source)
        current = source
        while current in link_dict:
            next_pseudo = link_dict[current]
            if next_pseudo in visited or next_pseudo == current:
                break
            chain.append(next_pseudo)
            visited.add(next_pseudo)
            current = next_pseudo
        chains.append(chain)

    reconstructed_trip_map = {}
    for i, chain in enumerate(chains):
        for pseudo_id in chain:
            reconstructed_trip_map[pseudo_id] = f"retrip_{i}"

    df_eval = df_trajet_simule[["pseudonym_id", "trip_id", "start_time"]].copy()
    df_eval["retrip_id"] = df_eval["pseudonym_id"].map(reconstructed_trip_map)

    # Métrique 1 : % de trajets entièrement réidentifiés
    fragmentation = df_eval.groupby("trip_id")["retrip_id"].nunique()
    fully_reidentified = (fragmentation == 1).sum()
    percent_fully = 100 * fully_reidentified / len(fragmentation)

    # Métrique 2 : IDF1 pairwise
    df_sorted = df_eval.sort_values(["trip_id", "start_time"])
    true_links = []
    for _, group in df_sorted.groupby("trip_id"):
        pseudos = group["pseudonym_id"].tolist()
        true_links.extend([(pseudos[i], pseudos[i+1]) for i in range(len(pseudos)-1)])

    pred_links = list(zip(df_links["source"], df_links["target"]))
    true_set = set(true_links)
    pred_set = set(pred_links)

    tp = len(true_set & pred_set)
    fp = len(pred_set - true_set)
    fn = len(true_set - pred_set)

    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    idf1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

    # Métrique 3 : ID switches
    id_switches = 0
    for _, group in df_eval.sort_values(["trip_id", "start_time"]).groupby("trip_id"):
        rids = group["retrip_id"].tolist()
        switches = sum([rids[i] != rids[i-1] for i in range(1, len(rids))])
        id_switches += switches

    # Métrique 4 : faux liens
    false_links = fp

    # Métrique 5 : ARI
    ari = adjusted_rand_score(df_eval["trip_id"], df_eval["retrip_id"].fillna('unlinked'))

    return pd.DataFrame([{
        "%_trips_fully_reidentified": percent_fully,
        "false_links": false_links,
        "id_switches": id_switches,
        "ARI": ari,
        "IDF1": 100 * idf1
    }])

df_eval = df_trajet_simule[["pseudonym_id", "trip_id"]].copy()
df_eval["retrip_id"] = df_eval["pseudonym_id"].map(reconstructed_trip_map)
df_eval["start_time"] = df_trajet_simule["start_time"]  # ajouter le temps pour les ordres
results = evaluate_linking_attack_pairwise(df_trajet_simule, df_links)
print(results.T)

                                     0
%_trips_fully_reidentified   21.504660
false_links                 201.000000
id_switches                  62.000000
ARI                           0.000052
IDF1                         46.153846


In [107]:
import pandas as pd
from geopy.distance import geodesic
from tqdm import tqdm

def relinking_score_kalman(row_A, row_B, weight_time=1, weight_space=1):
    time_gap = (row_B["start_time"] - row_A["end_time"]).total_seconds()
    if time_gap < 0:
        return float('inf')

    distance = geodesic(
        (row_A["end_latitude"], row_A["end_longitude"]),
        (row_B["start_latitude"], row_B["start_longitude"])
    ).meters

    return weight_time * time_gap + weight_space * distance

def evaluate_links(df_links, df_trajet_simule):
    from collections import defaultdict

    link_dict = dict(zip(df_links["source"], df_links["target"]))
    chains = []
    visited = set()

    for source in link_dict:
        if source in visited:
            continue
        chain = [source]
        visited.add(source)
        current = source
        while current in link_dict:
            next_pseudo = link_dict[current]
            if next_pseudo in visited or next_pseudo == current:
                break
            chain.append(next_pseudo)
            visited.add(next_pseudo)
            current = next_pseudo
        chains.append(chain)

    reconstructed_trip_map = {pseudo: f"retrip_{i}" for i, chain in enumerate(chains) for pseudo in chain}
    df_eval = df_trajet_simule[["pseudonym_id", "trip_id"]].copy()
    df_eval["retrip_id"] = df_eval["pseudonym_id"].map(reconstructed_trip_map)

    trip_fragmentation = df_eval.groupby("trip_id")["retrip_id"].nunique()
    complete_reid = (trip_fragmentation == 1).sum()
    complete_score = 100 * complete_reid / len(trip_fragmentation)

    true_links = []
    df_sorted = df_trajet_simule.sort_values(["trip_id", "start_time"])
    for _, group in df_sorted.groupby("trip_id"):
        pseudos = group["pseudonym_id"].tolist()
        true_links.extend([(pseudos[i], pseudos[i + 1]) for i in range(len(pseudos) - 1)])

    pred_links = list(zip(df_links["source"], df_links["target"]))
    true_set = set(true_links)
    pred_set = set(pred_links)
    tp = len(true_set & pred_set)
    fp = len(pred_set - true_set)
    fn = len(true_set - pred_set)

    precision = tp / (tp + fp) if tp + fp > 0 else 0
    recall = tp / (tp + fn) if tp + fn > 0 else 0
    idf1 = 2 * precision * recall / (precision + recall) if precision + recall > 0 else 0

    return complete_score, precision, recall, idf1

def grid_search_relinking(df_trajet_simule, n_top=5):
    results = []
    for weight_time in [0.5, 1, 2, 3]:
        for weight_space in [0.5, 1, 2, 3]:
            best_links = []
            for _, row_A in df_trajet_simule.iterrows():
                candidates = df_trajet_simule[df_trajet_simule["start_time"] > row_A["end_time"]]
                candidates = candidates.nsmallest(n_top, "start_time")
                best_score = float("inf")
                best_match_id = None
                for _, row_B in candidates.iterrows():
                    score = relinking_score_kalman(row_A, row_B, weight_time, weight_space)
                    if score < best_score:
                        best_score = score
                        best_match_id = row_B["pseudonym_id"]
                if best_score < 100:
                    best_links.append({
                        "source": row_A["pseudonym_id"],
                        "target": best_match_id,
                        "score": best_score
                    })

            df_links = pd.DataFrame(best_links)
            complete, prec, rec, idf1 = evaluate_links(df_links, df_trajet_simule)
            results.append({
                "weight_time": weight_time,
                "weight_space": weight_space,
                "Réidentification complète": round(complete, 2),
                "Precision": round(prec * 100, 2),
                "Recall": round(rec * 100, 2),
                "IDF1": round(idf1 * 100, 2)
            })

    return pd.DataFrame(results)

# Lancer la recherche et afficher les résultats
df_results = grid_search_relinking(df_trajet_simule)
print(df_results.sort_values("IDF1", ascending=False))


    weight_time  weight_space  Réidentification complète  Precision  Recall  \
15          3.0           3.0                      14.45      48.60   74.82   
14          3.0           2.0                      16.78      42.91   76.26   
11          2.0           3.0                      17.31      42.11   74.82   
10          2.0           2.0                      19.91      37.46   76.26   
7           1.0           3.0                      21.30      35.62   74.82   
13          3.0           1.0                      21.17      35.00   75.54   
9           2.0           1.0                      25.50      30.29   76.26   
6           1.0           2.0                      25.97      30.29   76.26   
3           0.5           3.0                      27.96      28.30   74.10   
12          3.0           0.5                      27.96      25.69   73.38   
5           1.0           1.0                      34.75      23.79   77.70   
2           0.5           2.0                      3